In [1]:
%run PilotForestAreas.ipynb

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
import param as pm
import panel as pn
import pandas as pd
import datetime as dt
import numpy as np
pn.extension()

In [3]:
f1.sale_value()

3118225.187825

In [4]:
f1.forested_area_hectares

7478

In [5]:
f1.acres_protected()

18470.66

In [6]:
f1.tons_carbon_per_year

21867.58

In [7]:
class Carbon(pm.Parameterized):
    forest = pm.Selector(default=f1, objects=[f1,f2])
    initial_carbon_price = pm.Number(29)
    carbon_price_appreciation = pm.Number(0.15, bounds=(0,1), step=0.01)
    stewardship_budget_increase = pm.Number(0.075, bounds=(0,1), step=0.01)
    tax_rate = pm.Number(0.2, bounds=(0,1), step=0.01)
    centree_admin_commission = pm.Number(0.15, bounds=(0,1), step=0.01)
    pfa_commission = pm.Number(0.425, bounds=(0,1), step=0.01)
    land_title_holders_commission = pm.Number(0.425, bounds=(0,1), step=0.01)
    below_soil_tonnage_factor = pm.Number(1, bounds=(0,2), step=0.01)
    
    def compute(self, periods = 10, freq = 'Y'):
        
        df = pd.DataFrame(index=pd.date_range(dt.datetime.now(), periods=periods, freq=freq))
        df['Carbon Price'] = [self.initial_carbon_price * (1+self.carbon_price_appreciation)**x for x in range(periods)]

        df['Gross Revenue'] = df['Carbon Price'] * self.forest.tons_carbon_per_year * (1 + self.below_soil_tonnage_factor)

        df['Net Revenue'] = df['Gross Revenue'] * (1 - self.tax_rate)

        df['Stewardship Budget'] = [self.forest.forested_area_hectares * self.forest.stewardship_rate_per_hectare * (1+self.stewardship_budget_increase)**x for x in range(periods)]

        df['Gross Profit'] = df['Net Revenue'] - df['Stewardship Budget']

        df['Centree (Admin/Commission)'] = df['Gross Profit'] * self.centree_admin_commission

        df['PFA Commission'] = df['Gross Profit'] * self.pfa_commission
        df['Land Title Holders Commission'] = df['Gross Profit'] * self.land_title_holders_commission

        df['PFA Holder Return as Percentage of Initial Cost'] = df['PFA Commission'] / self.forest.sale_value()
        
        df.index.name = "Date"

        return df
    
    def view_model_table(self):
        return self.compute().reset_index().hvplot.table()
    
    def view_model_normalized_chart(self):
        carbon = self.compute()
        carbon_normal = carbon / carbon.iloc[0] 
        carbon_normal = carbon_normal + np.random.randn(*carbon_normal.shape) / 10
        return pn.panel(carbon_normal.hvplot.line(title="Normalized Growth Projections of Carbon Revenue", height=340, width=1000))
    
    def view(self):
        return pn.Row(
            self, pn.Tabs(
                ("Carbon Model", self.view_model_table), 
                ("Normalized Projections", self.view_model_normalized_chart),
            ))

In [8]:
c1 = Carbon(forest=f1, below_soil_tonnage_factor=0, carbon_price_appreciation=0.15)

In [9]:
c1.view()

Row
    [0] Column(margin=5, name='Carbon', width=300)
        [0] StaticText(value='<b>Carbon</b>')
        [1] Row(width_policy='max')
            [0] Select(margin=(5, 0, 5, 10), name='Forest', options=OrderedDict([('Forest00786...]), value=Forest, width=240)
            [1] Toggle(align='end', button_type='primary', height_policy='fit', margin=(0, 0, 5, 10), max_height=30, max_width=20, name='⋮')
        [2] FloatInput(name='Initial carbon price', value=29)
        [3] FloatSlider(name='Carbon price a..., step=0.01, value=0.15)
        [4] FloatSlider(name='Stewardship b..., step=0.01, value=0.075)
        [5] FloatSlider(name='Tax rate', step=0.01, value=0.2)
        [6] FloatSlider(name='Centree admin c..., step=0.01, value=0.15)
        [7] FloatSlider(name='Pfa commission', step=0.01, value=0.425)
        [8] FloatSlider(name='Land title holders c..., step=0.01, value=0.425)
        [9] FloatSlider(end=2, name='Below soil tonnage f..., step=0.01)
    [1] Tabs
        [0] ParamMethod(method, _pane=HoloViews, name='Carbon Model')
        [1] ParamMethod(method, _pane=HoloViews, name='Normalized Projections')